In [2]:
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import scipy as sc
import cv2 as cv
import matplotlib.pyplot as plt
import rasterio as riot
from osgeo import gdal
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
Image.MAX_IMAGE_PIXELS = None



In [7]:
df = pd.read_csv('Farinotti_mean_thickness_RGI_ID_2.csv')

df = df[[
    'RGIId',
    'Farinotti Mean Thickness'
]]
df['region'] = df['RGIId'].str[6:8]
df = df.sort_values('RGIId')
df = df.dropna()



dfy = pd.DataFrame()
for region_number in range(1,20,1):
    if len(str(region_number)) == 1:
        N = 1
        (region_number) = str(region_number).zfill(N + len(str(region_number)))
    else:
        str(region_number) == str(region_number)
    dft = df[df['region'] == str(region_number)]
    
    extracted_count = len(dft)
    mean_thickness = dft['Farinotti Mean Thickness'].mean()
    median_thickness = dft['Farinotti Mean Thickness'].median()
    
    dfy = pd.concat([dfy, pd.Series(region_number, name = 'mean thickness')])
    dfy = dfy.reset_index()
    dfy = dfy.drop('index', axis = 1)
    # dfy = dfy.drop('0', axis = 1)
    dfy.loc[dfy.index[-1], 'region'] = region_number
    dfy.loc[dfy.index[-1], 'extracted mean'] = mean_thickness
    dfy.loc[dfy.index[-1], 'extracted median'] = median_thickness
    dfy.loc[dfy.index[-1], 'extracted count'] = extracted_count

dfy.loc[dfy.index[0], 'FR mean'] = 218
dfy.loc[dfy.index[1], 'FR mean'] = 72
dfy.loc[dfy.index[2], 'FR mean'] = 270
dfy.loc[dfy.index[3], 'FR mean'] = 210
dfy.loc[dfy.index[4], 'FR mean'] = 175
dfy.loc[dfy.index[5], 'FR mean'] = 341
dfy.loc[dfy.index[6], 'FR mean'] = 220
dfy.loc[dfy.index[7], 'FR mean'] = 101
dfy.loc[dfy.index[8], 'FR mean'] = 283
dfy.loc[dfy.index[9], 'FR mean'] = 56
dfy.loc[dfy.index[10], 'FR mean'] = 61
dfy.loc[dfy.index[11], 'FR mean'] = 48
dfy.loc[dfy.index[12], 'FR mean'] = 66
dfy.loc[dfy.index[13], 'FR mean'] = 85
dfy.loc[dfy.index[14], 'FR mean'] = 59
dfy.loc[dfy.index[15], 'FR mean'] = 42
dfy.loc[dfy.index[16], 'FR mean'] = 181
dfy.loc[dfy.index[17], 'FR mean'] = 63
dfy.loc[dfy.index[18], 'FR mean'] = 349


dfy.loc[dfy.index[0], 'FR area'] = 86677
dfy.loc[dfy.index[1], 'FR area'] = 14629
dfy.loc[dfy.index[2], 'FR area'] = 104920
dfy.loc[dfy.index[3], 'FR area'] = 40860
dfy.loc[dfy.index[4], 'FR area'] = 89651
dfy.loc[dfy.index[5], 'FR area'] = 11052
dfy.loc[dfy.index[6], 'FR area'] = 33932
dfy.loc[dfy.index[7], 'FR area'] = 2947
dfy.loc[dfy.index[8], 'FR area'] = 51551
dfy.loc[dfy.index[9], 'FR area'] = 2399
dfy.loc[dfy.index[10], 'FR area'] = 2091
dfy.loc[dfy.index[11], 'FR area'] = 1305
dfy.loc[dfy.index[12], 'FR area'] = 49295
dfy.loc[dfy.index[13], 'FR area'] = 33561
dfy.loc[dfy.index[14], 'FR area'] = 14734
dfy.loc[dfy.index[15], 'FR area'] = 2341
dfy.loc[dfy.index[16], 'FR area'] = 29368
dfy.loc[dfy.index[17], 'FR area'] = 1161
dfy.loc[dfy.index[18], 'FR area'] = 132771

dfy.loc[dfy.index[0], 'FR count'] = 27108
dfy.loc[dfy.index[1], 'FR count'] = 18862
dfy.loc[dfy.index[2], 'FR count'] = 4549
dfy.loc[dfy.index[3], 'FR count'] = 7422
dfy.loc[dfy.index[4], 'FR count'] = 19306
dfy.loc[dfy.index[5], 'FR count'] = 567
dfy.loc[dfy.index[6], 'FR count'] = 1615
dfy.loc[dfy.index[7], 'FR count'] = 3417
dfy.loc[dfy.index[8], 'FR count'] = 1069
dfy.loc[dfy.index[9], 'FR count'] = 5144
dfy.loc[dfy.index[10], 'FR count'] = 3927
dfy.loc[dfy.index[11], 'FR count'] = 1887
dfy.loc[dfy.index[12], 'FR count'] = 54429
dfy.loc[dfy.index[13], 'FR count'] = 27986
dfy.loc[dfy.index[14], 'FR count'] = 13119
dfy.loc[dfy.index[15], 'FR count'] = 2940
dfy.loc[dfy.index[16], 'FR count'] = 15908
dfy.loc[dfy.index[17], 'FR count'] = 3537
dfy.loc[dfy.index[18], 'FR count'] = 2751
dfy['FR mean'] = dfy['FR mean'].astype(int)

dfy['FR area'] = dfy['FR area'].astype(int)

dfy['FR count'] = dfy['FR count'].astype(int)
dfy['extracted count'] = dfy['extracted count'].astype(int)

dfy = dfy[[
    'region',
    'extracted mean',
    # 'extracted median',
    'FR mean',
    'FR area',
    'FR count',
    'extracted count'
]]
df = dfy


In [8]:
df['"avg" area'] = df['FR area'] / df['FR count']
df['residual'] = df['extracted mean'] - df['FR mean']
df['ratio'] = df['extracted mean'] / df['FR mean']
df.sort_values('ratio', ascending = False)

,region,extracted mean,FR mean,FR area,FR count,extracted count,"""avg"" area",residual,ratio
15,16,23.361094,42,2341,2940,2939,0.796259,-18.638906,0.556217
11,12,22.077926,48,1305,1887,1887,0.691574,-25.922074,0.459957
14,15,26.130335,59,14734,13119,13119,1.123104,-32.869665,0.442887
8,09,120.449643,283,51551,1069,1069,48.223573,-162.550357,0.425617
9,10,21.759381,56,2399,5144,5144,0.466369,-34.240619,0.388560
6,07,84.330272,220,33932,1615,1615,21.010526,-135.669728,0.383319
12,13,25.281591,66,49295,54429,54428,0.905675,-40.718409,0.383054
10,11,20.938068,61,2091,3927,3927,0.532468,-40.061932,0.343247
1,02,24.189294,72,14629,18862,18855,0.775581,-47.810706,0.335962
7,08,32.861840,101,2947,3417,3417,0.862452,-68.138160,0.325365


In [4]:
pth_2 = '/home/simonhans/data/prethicktor/RGI/rgi60-attribs/'
# pth_2 = '/data/fast1/glacierml/data/RGI/rgi60-attribs/'
RGI_extra = pd.DataFrame(columns = ['Farinotti Mean Thickness'])
for file in tqdm(os.listdir(pth_2)):
    file_reader = pd.read_csv(pth_2 + file, encoding_errors = 'replace', on_bad_lines = 'skip')
    RGI_extra = pd.concat([RGI_extra, file_reader], ignore_index = True)

    # select only RGI data that was used to train the model   
RGI = RGI_extra[[
'RGIId',
    'Farinotti Mean Thickness'
]]
print(RGI)

100%|██████████| 19/19 [00:02<00:00,  8.06it/s]

                 RGIId Farinotti Mean Thickness
0       RGI60-07.00001                      NaN
1       RGI60-07.00002                      NaN
2       RGI60-07.00003                      NaN
3       RGI60-07.00004                      NaN
4       RGI60-07.00005                      NaN
...                ...                      ...
216496  RGI60-17.15904                      NaN
216497  RGI60-17.15905                      NaN
216498  RGI60-17.15906                      NaN
216499  RGI60-17.15907                      NaN
216500  RGI60-17.15908                      NaN

[216501 rows x 2 columns]


In [174]:


pth_1 = '/home/simonhans/data/prethicktor/RGI/outlines/Farinotti/'
# rootdir = '~'

df1 = pd.DataFrame()
df2 = pd.DataFrame()
for region_number in range(1,20,1)[:1]:
            
    if len(str(region_number)) == 1:
        N = 1
        region_number = str(region_number).zfill(N + len(str(region_number)))
    else:
        str(region_number) == str(region_number)
        
    region_folder = pth_1 + 'RGI60-' + str(region_number) + '/'
    for file in tqdm(os.listdir(region_folder)):
        
        RGIId = file[:14]
        
        raster = gdal.Open(region_folder + file)
        got = raster.GetGeoTransform()
        # print(got)
        pixelSizeX = got[1]
        pixelSizeY = -got[5]
        pixel_area = (pixelSizeX * pixelSizeY)

        im = Image.open(region_folder + file)
        imarray = np.array(im)
        df = pd.DataFrame(imarray)
        df = df.replace(0.0, np.nan) 
        
        volume = df * pixel_area
        
        # print(volume_1)
        # print(volume_2)
        total_volume = np.nansum(volume)
        
        area = np.count_nonzero(~np.isnan(df)) * pixel_area

        # thickness_1 = total_volume_1 / area_1
        # thickness_2 = total_volume_2 / area_2
        
        
        # mean_volume = sum(volume) / area

        total_volume = pd.Series(total_volume)
        df1 = df1.append(total_volume, ignore_index = True)
        df1.loc[df1.index[-1], 'Area'] = area / 1e6
        df1.loc[df1.index[-1], 'RGIId'] = RGIId
    
df1.to_csv('Farinotti_vol.csv'

        
        
        
    #     break
    # break
        

#         break
#     break

100%|████████████████████████████████████| 27108/27108 [02:06<00:00, 213.61it/s]


In [175]:
df1

,0,Area,RGIId
0,3.497241e+05,28125.0,RGI60-01.09219
1,5.180656e+08,7483125.0,RGI60-01.20711
2,2.668925e+08,3992500.0,RGI60-01.09453
3,7.334464e+06,418125.0,RGI60-01.24136
4,1.482672e+08,2987500.0,RGI60-01.03796
...,...,...,...
27103,3.490575e+06,178125.0,RGI60-01.25906
27104,2.519825e+08,4052500.0,RGI60-01.04617
27105,5.329139e+06,285000.0,RGI60-01.24358
27106,1.834148e+08,3063125.0,RGI60-01.00660


In [178]:
f = (np.nansum(df1[0])) / 86677 / 1e6
# s = (np.nansum(df1['Vol 2'])) / 86677 / 1e6
sa = (np.nansum(df1['Area']))
print(f)

print('summed area = ' + str(sa / 1e6))
# print(s)
# r = np.nansum(df1['Area']) 
# q = np.nansum(df1['Area 2']) / 25 

# print(r)
# print(q)
# df7.mean()/86677
# print(df1)
# print('')
# print(df2)
# print('')

218.9561018767147
summed area = 86677.6875


71.05487514960733

In [182]:
df1['Area'] = df1['Area'] / 1e6
df1

,0,Area,RGIId,Thickness
0,3.497241e+05,0.028125,RGI60-01.09219,12.434633
1,5.180656e+08,7.483125,RGI60-01.20711,69.231179
2,2.668925e+08,3.992500,RGI60-01.09453,66.848477
3,7.334464e+06,0.418125,RGI60-01.24136,17.541319
4,1.482672e+08,2.987500,RGI60-01.03796,49.629183
...,...,...,...,...
27103,3.490575e+06,0.178125,RGI60-01.25906,19.596211
27104,2.519825e+08,4.052500,RGI60-01.04617,62.179526
27105,5.329139e+06,0.285000,RGI60-01.24358,18.698733
27106,1.834148e+08,3.063125,RGI60-01.00660,59.878322


In [183]:
df1['Thickness'] = df1[0] / df1['Area']

In [184]:
pth_2 = '/home/simonhans/data/prethicktor/RGI/rgi60-attribs/'
RGI_extra = pd.DataFrame(columns = ['Farinotti Mean Thickness'])
for file in tqdm(os.listdir(pth_2)):
    file_reader = pd.read_csv(pth_2 + file, encoding_errors = 'replace', on_bad_lines = 'skip')
    RGI_extra = pd.concat([RGI_extra, file_reader], ignore_index = True)

100%|███████████████████████████████████████████| 19/19 [00:02<00:00,  8.87it/s]


In [185]:
RGI_extra

,Farinotti Mean Thickness,RGIId,GLIMSId,BgnDate,EndDate,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,Slope,Aspect,Lmax,Status,Connect,Form,TermType,Surging,Linkages,Name
0,NaN,RGI60-07.00001,G017035E76705N,20080901.0,-9999999.0,17.0347,76.7052,7.0,1.0,0.597,142.0,459.0,268.0,16.7,204.0,1388.0,0.0,0.0,0.0,0.0,0.0,9.0,NaN
1,NaN,RGI60-07.00002,G016189E76787N,20080901.0,-9999999.0,16.1891,76.7872,7.0,1.0,0.581,234.0,582.0,343.0,18.3,277.0,1658.0,0.0,0.0,0.0,0.0,0.0,9.0,Bungebreen
2,NaN,RGI60-07.00003,G016392E76919N,20080901.0,-9999999.0,16.3915,76.9194,7.0,1.0,0.388,157.0,540.0,330.0,19.8,222.0,1200.0,0.0,0.0,0.0,0.0,0.0,9.0,Samarinbreen East
3,NaN,RGI60-07.00004,G015993E76942N,20080901.0,-9999999.0,15.9932,76.9423,7.0,1.0,1.044,665.0,933.0,799.0,13.8,62.0,1075.0,0.0,0.0,0.0,0.0,0.0,9.0,NaN
4,NaN,RGI60-07.00005,G016497E76956N,20080901.0,-9999999.0,16.4966,76.9564,7.0,1.0,0.500,186.0,396.0,263.0,17.7,35.0,1068.0,0.0,0.0,0.0,0.0,0.0,9.0,Mendelejevbreen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216496,NaN,RGI60-17.15904,G286335E46976S,20011103.0,-9999999.0,-73.6650,-46.9760,17.0,1.0,31.033,68.0,2192.0,1274.0,14.1,286.0,13890.0,0.0,0.0,0.0,2.0,9.0,9.0,Fraenkel
216497,NaN,RGI60-17.15905,G286325E46934S,20011103.0,-9999999.0,-73.6750,-46.9340,17.0,1.0,16.863,424.0,1614.0,1130.0,13.1,252.0,6356.0,0.0,0.0,0.0,0.0,9.0,9.0,Strindberg
216498,NaN,RGI60-17.15906,G286280E46914S,20011103.0,-9999999.0,-73.7200,-46.9140,17.0,1.0,1.430,721.0,1381.0,1193.0,19.8,231.0,1568.0,0.0,0.0,0.0,0.0,9.0,9.0,NaN
216499,NaN,RGI60-17.15907,G286285E46921S,20011103.0,-9999999.0,-73.7150,-46.9210,17.0,1.0,1.060,894.0,1338.0,1133.0,20.4,186.0,1279.0,0.0,0.0,0.0,0.0,9.0,9.0,NaN


In [33]:
sum(df1[0])

771970.8180069923

In [15]:
print('method 1 = ' + str(sum(df1[0]) / len[df1][0]))
print('method 2 = ' + str(sum(df2)))
print('max mean = ' + str(sum(df3)))
print('method 3 = ' + str(sum(df4)))
print('method 4 = ' + str(sum(df5)))

TypeError: 'builtin_function_or_method' object is not subscriptable

In [12]:
df1

,0
0,12.189878
0,64.344101
0,59.282726
0,17.252831
0,45.369186
...,...
0,17.867540
0,56.909782
0,17.447773
0,46.985695


In [14]:
pth_1 = '/home/simonhans/data/prethicktor/RGI/outlines/Millan/'
# rootdir = '~'
dft = pd.DataFrame()
for region_number in range(1,3,1)[:1]:
    print(region_number)
    # if len(str(region_number)) == 1:
    #     N = 1
    #     region_number = str(region_number).zfill(N + len(str(region_number)))
    # else:
    #     str(region_number) == str(region_number)
        
    region_folder = pth_1 + 'RGI-' + str(region_number) + '/'
    for file in (os.listdir(region_folder)[1:2]):

        print(file)
        im = Image.open(region_folder + file)
        im.show()
        imarray = np.array(im)
#         print(imarray.shape)
        df = pd.DataFrame(imarray)
        # print(df.head())
        df = df.replace(0.0, np.nan)
#         # print(df)

#         # mean_glacier_thickness_1 = df.mean().mean()

        mean_glacier_thickness_2 = pd.Series(np.nanmean(df.to_numpy()), name = 'thickness')
        dft = pd.concat(
            [dft, mean_glacier_thickness_2]
        )
#         # mean_by_hand = np.nansum(df) / np.count_nonzero(
#         #             ~np.isnan(df))

#         # print('mean by hand = ' + str(mean_by_hand))
#         # print('df.mean().mean() = ' + str(mean_glacier_thickness_1))
        # print('np.nanmean(df.to_numpy()) = ' + str(mean_glacier_thickness_2))
#         print('')
        # break
    #     break
    # break


# mean_glacier_thickness_1 = df.mean().mean()

# mean_glacier_thickness_2 = np.nanmean(df.to_numpy())

# mean by hand = np.nansum(
#     mean_glacier_thickness) / np.count_nonzero(~np.isnan(mean_glacier_thickness))

        
#         RGI['Farinotti Mean Thickness'].loc[RGI['RGIId'] == file[:14]] = mean_glacier_thickness
    #     break
    # break

1
THICKNESS_RGI-1.2_2021July01.tif


In [15]:
mean_glacier_thickness_2

81.16746

In [6]:
dft

,0
0,81.167458
0,130.740326
0,111.159630
0,210.449890
0,52.275787
0,129.307739
0,37.871696
0,193.966614
0,68.790810


In [5]:
dft.mean() * 101

0    11398.748047
dtype: float32